In [2]:
# pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
import transformers as t
import torch
import peft
import time
model_dir = "./llama/llama-2-7b-chat-hf"

tokenizer = t.AutoTokenizer.from_pretrained(model_dir)
model = t.AutoModelForCausalLM.from_pretrained(model_dir)


/Users/serena/miniconda3/envs/llama2_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.58s/it]


In [13]:
import safetensors

pt_state_dict = safetensors.torch.load_file("./trained/llama-2-7b-chat-hf/mt0-large-lora/checkpoint-43/adapter_model.safetensors")
torch.save(pt_state_dict, "adapter_model.bin")


In [3]:
tokenizer.pad_token_id = 0
config = peft.LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"], lora_dropout=0.005, bias="none", task_type="CAUSAL_LM")
model = peft.get_peft_model(model, config)
peft.set_peft_model_state_dict(model, torch.load("./adapter_model.bin"))
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model = model.to(device)
print('device', device)

device mps


In [4]:
# TEMPLATE = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"
# INSTRUCTION = "Python how to insert something at the beginning of a list?"
prompt = 'How do I know if I have ADHD or autism?'
pipe = t.pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, device='mps')
print("pipe(prompt)", pipe(prompt))

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

pipe(prompt) [{'generated_text': 'How do I know if I have ADHD or autism?\n<h2>How Do I Know If I Have ADHD or Autism?</h2>\n\nIf you are concerned about your mental health, it is essential to consult with a qualified healthcare professional. They can provide a proper evaluation and diagnosis based on your symptoms and medical history. Here are some signs that may indicate ADHD or autism:\n\n1. Difficulty paying attention: If you find it challenging to focus on tasks, follow instructions, or complete tasks, you may have ADHD.\n\n2. Impulsivity: If you often act without thinking, interrupt others, or have difficulty waiting for your turn, you may have ADHD.\n\n3. Hyperactivity: If you feel restless, fidgety, or have a lot of energy, you may have ADHD.\n\n4. Difficulty with social interactions: If'}]
